# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pymorphy2
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vaceslavefimov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vaceslavefimov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import re

In [4]:
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import requests

# Загрузка данных

In [6]:
# возьмем обработанный датасет из предыдущей лабораторной работы
df = pd.read_csv('LW4/jobs.csv')[['text', 'normalized_text']]

df['normalized_text'] = df['normalized_text'].map(lambda tokens: ' '.join(eval(tokens)))

# удалим пустые документы
df = df[df['normalized_text'].map(lambda tokens: len(tokens)) > 0].reset_index(drop=True)
df

,text,normalized_text
0,"ребят, привет! заранее извиняюсь, если пишу не...",ребята привет заранее извиняться писать нужный...
1,привет!\n*позиция:* team lead (data science)\n...,привет позиция компания лаборатория искусствен...
2,всем привет!\nищу в свою команду big data инже...,весь привет искать команда инженер компания пл...
3,"эта вакансия для тех, кто хочет *решать соревн...",этот вакансия тот хотеть решать соревнование р...
4,*компания:* сбербанк\n*вакансия:* data scienti...,компания сбербанк вакансия город москва вилка ...
...,...,...
3740,дружественная нам исследовательская группа нан...,дружественный мы исследовательский группа нани...
3741,"мопед не мой, так что все вопросы по указанном...",мопед вопрос указанный коллега попросить отпра...
3742,"отношения никакого к университету не имею, но ...",отношение никакой университет иметь решить под...
3743,всем привет!\nу нас в гренобльском офисе crite...,весь привет гренобльский офис открыться позици...


# Обработка текста

In [8]:
def check_spelling(text: str):
    """
    Исправляет опечатки слов в тексте с помощью Yandex Spellchecker API
    param text: текст, в котором нужно исправить опечатки
    """
    
    domain = "https://speller.yandex.net/services/spellservice.json"
    words = text.split()
    if len(words) == 1:
        request = requests.get(domain + "/checkText?text=" + words[0])
        if request.json():
            return request.json()[0]["word"], request.json()[0]["s"]
        else:
            return None
    
    elif len(words) > 1:
        words = "+".join(words)
        request = requests.get(domain + "/checkText?text=" + words)
        if request.json():
            response = [(i["word"], i["s"]) for i in request.json()]
            return response
        else:
            return None
    return None


def edit_spelling(query: str) -> str:
    """
    Редактирует текст, исправляя в нем опечатки. В случае если не удалось найти в корпусе Yandex заданное слово,
    оно не изменяется.
    param query: текст для редактиврования
    """
    
    edited_words = []
    words = query.split()
    for word in words:
        edited_word = check_spelling(word)
        edited_words.append(word if edited_word is None else edited_word[1][0])
    return ' '.join(edited_words)

In [9]:
def text_to_wordlist(text: str) -> list:
    """
    Преобразует текст в список слов, удаляя в нем символы пунктуации, цифры и другие лишние символы
    param text: текст для преобразования
    """
    
    text = re.sub(r"http[s]?://(?:[а-яА-Я]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", text)
    text = re.sub("[^а-яА-Я]"," ", text)
    return text.lower().split()


def remove_stopwords(words: list) -> list:
    """
    Удаляет стоп-слова
    param words: список слов, в которых нужно удалить стоп-слова
    """
    
    return [w for w in words if not w in stopwords.words('russian')]


def process_text(text: str) -> str:
    """
    param text: токенизируемый текст
    """
    
    raw_texts = tokenizer.tokenize(text.strip())
    texts = []
    for raw_text in raw_texts:
        if len(raw_text) > 0:
            texts.append(remove_stopwords(text_to_wordlist(raw_text)))
    return [item for sublist in texts for item in sublist]


def normalize(token: str) -> str:
    """
    Нормализация токена
    param token: токен
    """
    
    return morph.parse(token)[0].normal_form

In [52]:
def process_query(query: str) -> str:
    """
    Редактирование запроса. Включает в себя следующие шаги:
    1. Исправление опечаток
    2. Токенизация
    3. Удаление стоп-слов
    4. Нормализация
    """
    return ' '.join(list(map(lambda word: normalize(word), process_text(edit_spelling(query)))))

In [11]:
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
morph = pymorphy2.MorphAnalyzer()

# Информационный поиск

In [7]:
def cosine(a: list, b: list) -> np.float64:
    """
    Считает косинусное расстояние между векторами
    param a, b: векторы
    """
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [53]:
def recommend(corpus: pd.Series, query: str, top_n: int = 10) -> list:
    """
    Возвращает наиболее похожие тексты на основании запроса
    param df: датафрейм, содержащий тексты
    param query: текст, для которого нужно сделать рекомендацию. Возвращает список индексов в датасете df
    param top_n: количество рекомендуемых текстов
    """
    
    query = process_query(query)
    
    vectorizer = TfidfVectorizer()
    vectorized_texts = list(vectorizer.fit_transform(list(corpus) + [query]).toarray())
    vectorized_corpus = vectorized_texts[:-1]
    vectorized_query = vectorized_texts[-1]
    
    recommendations = sorted(enumerate(vectorized_corpus), key=lambda text: cosine(text[1], vectorized_query))
    return [index for index, text in recommendations[-top_n:]]

In [60]:
query = 'Привет! Нам нушен в старт-ап классныъ разрабачик данных'
recommendation_indices = recommend(df['normalized_text'], query)
df.loc[recommendation_indices]

,text,normalized_text
672,"всем привет!\nесть здесь ребята из минска, кот...",весь привет ребята минск который начинать свой...
333,всем привет!\nмы в ringlabs в киев очень ищем ...,весь привет киев очень искать наш отдел задача...
460,"всем привет!\nпитер, нужен nlp - специалист!\n...",весь привет питер нужный специалист ссылка вак...
2139,:lamoda: команда r&amp;d lamoda ~продала много...,команда продать трусель очень вести прошлый го...
3395,:lamoda: команда r&amp;d lamoda ~продала много...,команда продать трусель очень вести прошлый го...
517,:v:всем привет! у нас еще актуальна вакансия!\...,весь привет актуальный вакансия искать собрать...
3230,"друзья, привет!\n\nкомпания :ice_hockey_stick_...",друг привет компания искать собрать команда пр...
2070,"*midlle/senior analyst, operation efficiency*\...",компания лента ленточка локация москва зарплат...
2275,добрый день! \n\n\nдолжность: data analyst\nко...,добрый день должность компания лента локация м...
1762,в молодой амбициозный старт-ап <http://fabriqu...,молодой амбициозный старт ап требоваться созда...
